# Basic usage of the ``fakecbed`` library #

## A NOTE BEFORE STARTING ##

Since the ``fakecbed`` git repository tracks this notebook under its original
basename ``basic_usage.ipynb``, we recommend that you copy the original notebook
and rename it to any other basename that is not one of the original basenames
that appear in the ``<root>/examples`` directory before executing any of the
notebook cells below, where ``<root>`` is the root of the ``fakecbed``
repository. This way you can explore the notebook by executing and modifying
cells without changing the original notebook, which is being tracked by git.

## Import necessary modules ##

In [ ]:
# For general array handling.
import numpy as np

# For creating distortion models.
import distoptica



# The library that is the subject of this demonstration.
import fakecbed

In [ ]:
%matplotlib ipympl
%matplotlib ipympl

## Introduction ##

In this notebook, we demonstrate how one can use each function and class in the
``fakecbed`` library.

You can find the documentation for the ``fakecbed`` library
[here](https://mrfitzpa.github.io/fakecbed/_autosummary/fakecbed.html).  It is
recommended that you consult the documentation of this library as you explore
the notebook. Moreover, users should execute the cells in the order that they
appear, i.e. from top to bottom, as some cells reference variables that are set
in other cells above them.

## Using the ``fakecbed`` library ##

Let's create a discretized fake convergent beam electron diffraction (CBED)
pattern step by step.

First, let's specify the number of pixels across the discretized CBED
pattern. Note that in ``fakecbed``, it is assumed that the number of pixels in
the discretized CBED pattern from left to right is equal to the number of pixels
in the discretized CBED pattern from top to bottom.

In [ ]:
num_pixels_across_pattern = 512

Next, let's specify the intensity pattern of the undistorted thermal diffuse
scattering (TDS) model.

In [ ]:
kwargs = {"center": (0.480, 0.490),
          "widths": (0.060, 0.050, 0.070, 0.055),
          "rotation_angle": np.pi/3,
          "val_at_center": 50,
          "functional_form": "asymmetric_gaussian"}
undistorted_tds_peak_0 = fakecbed.shapes.Peak(**kwargs)

kwargs = {"center": (0.50, 0.51),
          "widths": (0.075, 0.060, 0.045, 0.055),
          "rotation_angle": np.pi,
          "val_at_center": 55,
          "functional_form": "asymmetric_lorentzian"}
undistorted_tds_peak_1 = fakecbed.shapes.Peak(**kwargs)



kwargs = {"peaks": (undistorted_tds_peak_0, undistorted_tds_peak_1),
          "constant_bg": 3}
undistorted_tds_model = fakecbed.tds.Model(**kwargs)

All public classes from the modules ``fakecbed.shapes`` and ``fakecbed.tds``,
that are subclasses of ``fakecbed.shapes.BaseShape`` represent intensity
patterns of undistorted shapes. For any instance of any such class, one can
evaluate the intensity pattern of the undistorted shape being represented at
fractional horizontal and vertical coordinates of any set of points in a
hypothetical undistorted image.

In [ ]:
u_x = np.array(((0.30, 0.50, 0.60), 
                (0.50, 0.50, 0.45)))
u_y = np.array(((0.50, 0.55, 0.60), 
                (0.45, 0.40, 0.45)))

print(undistorted_tds_peak_0.eval(u_x, u_y))
print(undistorted_tds_peak_1.eval(u_x, u_y))
print(undistorted_tds_model.eval(u_x, u_y))

Note that all public classes in the library ``fakecbed`` are subclasses of
``fancytypes.PreSerializableAndUpdatable``, meaning that any instance of any
such subclass, except for ``fakecbed.shapes.BaseShape``, is pre-serializable,
can be constructed from a serializable representation, and has an updatable
subset of attributes. See
[here](https://mrfitzpa.github.io/fancytypes/_autosummary/fancytypes.PreSerializableAndUpdatable.html)
for a definition of pre-serialization, and the documentation for all the
attributes and methods associated with the class
``fancytypes.PreSerializableAndUpdatable``.

With our TDS model, we can begin to construct our discretized CBED pattern.

In [ ]:
kwargs = {"num_pixels_across_pattern": num_pixels_across_pattern,
          "undistorted_tds_model": undistorted_tds_model}
cbed_pattern = fakecbed.discretized.CBEDPattern(**kwargs)

We can visualize the discretized CBED pattern as we update it, using the
``signal`` attribute, which stores a ``hyperspy`` signal representation of the
fake CBED pattern. See
[here](https://mrfitzpa.github.io/fakecbed/_autosummary/fakecbed.discretized.CBEDPattern.html#fakecbed.discretized.CBEDPattern.signal)
for a description of the signal representation of the fake CBED pattern.  Note
that the image of the discretized CBED pattern is always automatically min-max
normalized.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 0.3,
          "cmap": "jet"}
cbed_pattern.signal.plot(**kwargs)

Next, let's add some CBED disks to our fake CBED pattern. In ``fakecbed``, we
assume that the undistorted CBED disks are either circular or elliptical. In
this demo, we will assume that the CBED disks are only circular for
simplicity. Furthermore, for simplicity sake again, we will only specify five
CBED disks, one of which will be absent in the image of the fake CBED pattern.

In [ ]:
undistorted_disk_radius = 1/20



kwargs = {"center": (0.500, 0.500),
          "radius": undistorted_disk_radius,
          "intra_shape_val": 1}
undistorted_disk_0_support = fakecbed.shapes.Circle(**kwargs)

kwargs["center"] = (0.300, 0.300)
undistorted_disk_1_support = fakecbed.shapes.Circle(**kwargs)

kwargs["center"] = (0.400, 0.980)
undistorted_disk_2_support = fakecbed.shapes.Circle(**kwargs)

kwargs["center"] = (0.400, 0.910)
undistorted_disk_3_support = fakecbed.shapes.Circle(**kwargs)

kwargs["center"] = (2, 2)
undistorted_disk_4_support = fakecbed.shapes.Circle(**kwargs)



kwargs = {"center": undistorted_disk_0_support.core_attrs["center"],
          "radius": undistorted_disk_radius,
          "intra_shape_val": 50}
circle = fakecbed.shapes.Circle(**kwargs)

kwargs = {"amplitude": 10, 
          "wavelength": 1/40,
          "propagation_direction": 7*np.pi/8,
          "phase": 0}
plane_wave = fakecbed.shapes.PlaneWave(**kwargs)

kwargs = {"center": undistorted_disk_0_support.core_attrs["center"],
          "semi_major_axis": 1.0*undistorted_disk_radius,
          "eccentricity": 0.9, 
          "rotation_angle": np.pi/4, 
          "intra_shape_val": -50}
ellipse = fakecbed.shapes.Ellipse(**kwargs)

intra_support_shapes = (ellipse, circle, plane_wave)

kwargs = {"support": undistorted_disk_0_support,
          "intra_support_shapes": intra_support_shapes}
undistorted_disk_0 = fakecbed.shapes.NonuniformBoundedShape(**kwargs)



kwargs = {"center": undistorted_disk_1_support.core_attrs["center"],
          "principal_quantum_number": 3,
          "azimuthal_quantum_number": 1, 
          "magnetic_quantum_number": 0,
          "effective_size": undistorted_disk_radius/10, 
          "renormalization_factor": 1e-2, 
          "rotation_angle": 2*np.pi/3}
orbital = fakecbed.shapes.Orbital(**kwargs)

intra_support_shapes = (orbital,)

kwargs = {"support": undistorted_disk_1_support,
          "intra_support_shapes": intra_support_shapes}
undistorted_disk_1 = fakecbed.shapes.NonuniformBoundedShape(**kwargs)



kwargs = {"center": undistorted_disk_2_support.core_attrs["center"],
          "radius": undistorted_disk_radius,
          "intra_shape_val": 5}
bg_ellipse = fakecbed.shapes.Circle(**kwargs)  # All circles are ellipses.

ellipse_center = (undistorted_disk_2_support.core_attrs["center"][0]-0.01,
                  undistorted_disk_2_support.core_attrs["center"][1])
kwargs = {"center": ellipse_center,
          "radius": undistorted_disk_radius,
          "intra_shape_val": 1}
fg_ellipse = fakecbed.shapes.Circle(**kwargs)

kwargs = {"fg_ellipse": fg_ellipse,
          "bg_ellipse": bg_ellipse}
lune = fakecbed.shapes.Lune(**kwargs)

kwargs = {"center": undistorted_disk_2_support.core_attrs["center"],
          "radius": undistorted_disk_radius,
          "intra_shape_val": 2}
circle = fakecbed.shapes.Circle(**kwargs)

intra_support_shapes = (lune, circle)

kwargs = {"support": undistorted_disk_2_support,
          "intra_support_shapes": intra_support_shapes}
undistorted_disk_2 = fakecbed.shapes.NonuniformBoundedShape(**kwargs)



kwargs = {"center": undistorted_disk_3_support.core_attrs["center"],
          "radius": undistorted_disk_radius,
          "intra_shape_val": 5}
circle = fakecbed.shapes.Circle(**kwargs)

intra_support_shapes = (circle,)

kwargs = {"support": undistorted_disk_3_support,
          "intra_support_shapes": intra_support_shapes}
undistorted_disk_3 = fakecbed.shapes.NonuniformBoundedShape(**kwargs)



kwargs = {"center": undistorted_disk_4_support.core_attrs["center"],
          "radius": undistorted_disk_radius,
          "intra_shape_val": 5}
circle = fakecbed.shapes.Circle(**kwargs)

kwargs = {"support": undistorted_disk_4_support,
          "intra_support_shapes": intra_support_shapes}
undistorted_disk_4 = fakecbed.shapes.NonuniformBoundedShape(**kwargs)



undistorted_disks = (undistorted_disk_0, 
                     undistorted_disk_1, 
                     undistorted_disk_2, 
                     undistorted_disk_3, 
                     undistorted_disk_4)



new_core_attr_subset_candidate = {"undistorted_disks": undistorted_disks}
cbed_pattern.update(new_core_attr_subset_candidate)

Let's visualize the updated fake CBED pattern.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 0.3,
          "cmap": "jet"}
cbed_pattern.signal.plot(**kwargs)

Next, let's add some miscellaneous shapes to our fake CBED pattern.

In [ ]:
undistorted_disk_radius = 1/20



kwargs = {"end_pt_1": (0.2, -0.05),
          "end_pt_2": (1.05, 0.60),
          "width": 0.03,
          "intra_shape_val": 2}
undistorted_misc_shape_0 = fakecbed.shapes.Band(**kwargs)



radial_range = (0.6*undistorted_disk_radius, 0.7*undistorted_disk_radius)

kwargs = {"center": (0.2, 0.8),
          "midpoint_angle": 5*np.pi/4,
          "subtending_angle": np.pi/3,
          "radial_range": radial_range,
          "intra_shape_val": 8}
undistorted_misc_shape_1 = fakecbed.shapes.Arc(**kwargs)



undistorted_misc_shapes = (undistorted_misc_shape_0, 
                           undistorted_misc_shape_1)



new_core_attr_subset_candidate = {"undistorted_misc_shapes": \
                                  undistorted_misc_shapes}
cbed_pattern.update(new_core_attr_subset_candidate)

Let's visualize the updated fake CBED pattern.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 0.3,
          "cmap": "jet"}
cbed_pattern.signal.plot(**kwargs)

Next, let's specify an illumination support that does not cover the entire
imaged fake CBED pattern.

In [ ]:
radial_reference_pt = undistorted_disk_0_support.core_attrs["center"]

kwargs = {"radial_reference_pt": radial_reference_pt,
          "radial_amplitudes": (0.55, 0.08, 0.07),
          "radial_phases": (0.00, 3*np.pi/5),
          "intra_shape_val": 1}
undistorted_outer_illumination_shape = fakecbed.shapes.GenericBlob(**kwargs)



new_core_attr_subset_candidate = {"undistorted_outer_illumination_shape": \
                                  undistorted_outer_illumination_shape}
cbed_pattern.update(new_core_attr_subset_candidate)

Let's visualize the updated fake CBED pattern.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 0.3,
          "cmap": "jet"}
cbed_pattern.signal.plot(**kwargs)

Next, let's specify a distortion model to use to distort the fake CBED pattern.

In [ ]:
center = (0.52, 0.49)

quadratic_radial_distortion_amplitude = -0.1

spiral_distortion_amplitude = 0.1

amplitude = 0.07
phase = 7*np.pi/8
elliptical_distortion_vector = (amplitude*np.cos(2*phase).item(), 
                                amplitude*np.sin(2*phase).item())

amplitude = 0.1
phase = 4*np.pi/3
parabolic_distortion_vector = (amplitude*np.cos(phase), 
                               amplitude*np.sin(phase))



kwargs = \
    {"center": \
     center,
     "quadratic_radial_distortion_amplitude": \
     quadratic_radial_distortion_amplitude,
     "elliptical_distortion_vector": \
     elliptical_distortion_vector,
     "spiral_distortion_amplitude": \
     spiral_distortion_amplitude,
     "parabolic_distortion_vector": \
     parabolic_distortion_vector}
standard_coord_transform_params = \
    distoptica.StandardCoordTransformParams(**kwargs)



kwargs = {"max_num_iterations": 20,
          "initial_damping": 1e-3,
          "factor_for_decreasing_damping": 9,
          "factor_for_increasing_damping": 11,
          "improvement_tol": 0.1, 
          "rel_err_tol": 1e-2, 
          "plateau_tol": 1e-3, 
          "plateau_patience": 2, 
          "skip_validation_and_conversion": False}
least_squares_alg_params = distoptica.LeastSquaresAlgParams(**kwargs)



kwargs = {"standard_coord_transform_params": standard_coord_transform_params,
          "sampling_grid_dims_in_pixels": 2*(num_pixels_across_pattern,),
          "device_name": "cpu",
          "least_squares_alg_params": least_squares_alg_params}
distortion_model = distoptica.generate_standard_distortion_model(**kwargs)



new_core_attr_subset_candidate = {"distortion_model": distortion_model}
cbed_pattern.update(new_core_attr_subset_candidate)

Let's visualize the updated fake CBED pattern.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 0.3,
          "cmap": "jet"}
cbed_pattern.signal.plot(**kwargs)

Next, let's add blur effects to the fake CBED pattern.

In [ ]:
new_core_attr_subset_candidate = {"gaussian_filter_std_dev": 2}
cbed_pattern.update(new_core_attr_subset_candidate)

Let's visualize the updated fake CBED pattern.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 0.3,
          "cmap": "jet"}
cbed_pattern.signal.plot(**kwargs)

Next, let's shot noise to the fake CBED pattern.

In [ ]:
new_core_attr_subset_candidate = {"apply_shot_noise": True}
cbed_pattern.update(new_core_attr_subset_candidate)

Let's visualize the updated fake CBED pattern.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 0.3,
          "cmap": "jet"}
cbed_pattern.signal.plot(**kwargs)

Next, let's add a detector partition to the fake CBED pattern.

In [ ]:
new_core_attr_subset_candidate = {"detector_partition_width_in_pixels": 4}
cbed_pattern.update(new_core_attr_subset_candidate)

Let's visualize the updated fake CBED pattern.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 0.3,
          "cmap": "jet"}
cbed_pattern.signal.plot(**kwargs)

Next, let's add a mask frame.

In [ ]:
new_core_attr_subset_candidate = {"mask_frame": (40, 20, 0, 60)}
cbed_pattern.update(new_core_attr_subset_candidate)

Let's visualize the updated fake CBED pattern.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 0.3,
          "cmap": "jet"}
cbed_pattern.signal.plot(**kwargs)

Lastly, to complete the fake CBED pattern, let's add some cold pixels.

In [ ]:
cold_pixels = ((245, 260), 
               (346, 150), 
               (250, 252))

new_core_attr_subset_candidate = {"cold_pixels": cold_pixels}
cbed_pattern.update(new_core_attr_subset_candidate)

Let's visualize the updated fake CBED pattern.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 0.3,
          "cmap": "jet"}
cbed_pattern.signal.plot(**kwargs)

With the fake CBED pattern complete, let's look at the metadata of its signal
representation. Among other items, it stores a pre-serialized version of the
core attributes of the fake CBED pattern.

In [ ]:
cbed_pattern.signal.metadata

Let's look at the remaining instance attributes of the fake CBED pattern. Note
that for each attribute mentioned below, except for ``device``, ``num_disks``,
and ``image_has_been_overridden``, there are multiple ways to access it using
public methods. Let ``<attr_name>`` be the name of such an attribute. We can
access attribute ``<attr_name>`` via ``cbed_pattern.<attr_name>``,
``cbed_pattern.get_<attr_name>(deep_copy=True)``, and
``cbed_pattern.get_<attr_name>(deep_copy=False)``, where the first two options
yield a deep copy, and the last option yields a reference. Returning a reference
may be preferred in some circumstances for faster data access and reduced memory
overhead. In whatever case, users should treat any such attribute as read-only
always.

The attribute ``signal`` can be accessed in the ways described in the previous
paragraph, i.e. via ``cbed_pattern.signal`` and the method
``cbed_pattern.get_signal``.

The first of the remaining attributes is actually a dictionary which stores the
so-called "core attributes". See
[here](https://mrfitzpa.github.io/fancytypes/_autosummary/fancytypes.Checkable.html)
for a discussion on core attributes.

In [ ]:
cbed_pattern.core_attrs

Next, we have the device on which computationally intensive PyTorch operations
are performed and attributes of the type ``torch.Tensor`` are stored.

In [ ]:
cbed_pattern.device

Next, we have the total number of CBED disks defined in the fake CBED
pattern. Note that this is not necessarily equal to the number of CBED disks
that appear in the image of the fake CBED pattern. In our current case, we
defined one of the CBED disks to be positioned outside of the field of view of
the image of the fake CBED pattern.

In [ ]:
cbed_pattern.num_disks

Next, we have the disk absence registry, which records which CBED disks are
absent from the image of the fake CBED pattern. In our current case, only the
last CBED disk defined is absent.

In [ ]:
cbed_pattern.disk_absence_registry

Next, we have the disk clipping registry, which records which CBED disks are
clipped in the image of the fake CBED pattern. Note that CBED disks that are
absent are considered clipped. In our current case, assuming a zero-based
indexing scheme, CBED disks #2 and #4 are clipped.

In [ ]:
cbed_pattern.disk_clipping_registry

Next, we have the image of the fake CBED pattern. This attribute stores the same
numerical data as ``cbed_pattern.signal.data[0]``, except as a ``PyTorch``
tensor.

In [ ]:
cbed_pattern.image

We can visualize the image as follows:

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 0.3,
          "cmap": "jet"}
cbed_pattern.signal.inav[0].plot(**kwargs)

Next, we have the illumination support of the fake CBED pattern. This attribute
stores the same numerical data as ``cbed_pattern.signal.data[1]``, except as a
``PyTorch`` tensor.

In [ ]:
cbed_pattern.illumination_support

We can visualize the illumination support masked by the mask frame as follows:

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 0.3,
          "cmap": "jet"}
cbed_pattern.signal.inav[1].plot(**kwargs)

Next, we have the disk overlap support of the fake CBED pattern. This attribute
stores the same numerical data as ``cbed_pattern.signal.data[2]``, except as a
``PyTorch`` tensor.

In [ ]:
cbed_pattern.disk_overlap_map

We can visualize the disk overlap map as follows:

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 0.3,
          "cmap": "jet"}
cbed_pattern.signal.inav[2].plot(**kwargs)

Next, we have the disk supports of the fake CBED pattern. This attribute stores
the same numerical data as ``cbed_pattern.signal.data[3:]``, except as a
``PyTorch`` tensor.

In [ ]:
cbed_pattern.disk_supports

We can visualize the disk supports as follows:

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 0.3,
          "cmap": "jet"}
cbed_pattern.signal.inav[3:].plot(**kwargs)

Lastly, we have the attribute ``image_has_been_overridden``, which indicates
whether or not the image of the fake CBED pattern has been overridden. In our
current case, the image has not yet been overridden:

In [ ]:
cbed_pattern.image_has_been_overridden

One can override the image and then subsequently reapply the masks implied by
the illumination support and cold pixels using the method
``override_image_then_reapply_mask``.

In [ ]:
N = cbed_pattern.image.shape[0]

overriding_image = np.ones((N, N))
overriding_image[:N//2, :] *= 2

cbed_pattern.override_image_then_reapply_mask(overriding_image)

We should find that the attribute ``image_has_been_overridden`` has been updated
accordingly.

In [ ]:
cbed_pattern.image_has_been_overridden

We can visualize the overridden image by plotting the signal representation of
the fake CBED pattern again.

In [ ]:
kwargs = {"axes_off": False, 
          "scalebar": False, 
          "colorbar": False, 
          "gamma": 0.3,
          "cmap": "jet"}
cbed_pattern.signal.plot(**kwargs)